In [2]:
import pandas as pd
import numpy as np

In [5]:
df_made_save = pd.read_csv('save2data.csv')
df_made_save = df_made_save.drop_duplicates(subset=['gID']).reset_index(drop=True)

In [6]:
df_made_save.head(10)

,gID,scoreDiff,inning,Current Outs,onFirst,onSecond,onThird,pID,name
0,ANA201004050,3,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
1,ANA201004230,2,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
2,ANA201004260,3,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
3,ANA201006220,3,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
4,ANA201006230,1,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
5,ANA201006260,2,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
6,ANA201006290,1,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
7,ANA201007010,1,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
8,ANA201007160,1,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
9,ANA201007170,1,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes


In [7]:
df_last_pitcher = pd.read_csv('last_pitcher.csv')
df_last_pitcher.head(10)

,GAME_ID,AWAY_TEAM_ID,INN_CT,BAT_HOME_ID,OUTS_CT,BALLS_CT,STRIKES_CT,PITCH_SEQ_TX,AWAY_SCORE_CT,HOME_SCORE_CT,...,RUN2_FATE_ID,RUN3_FATE_ID,FATE_RUNS_CT,ASS6_FLD_CD,ASS7_FLD_CD,ASS8_FLD_CD,ASS9_FLD_CD,ASS10_FLD_CD,UNKNOWN_OUT_EXC_FL,UNCERTAIN_PLAY_EXC_FL
0,ANA201004050,MIN,9,0,0,1,2,.BCFS,3,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F,F
1,ANA201004050,MIN,8,1,0,1,1,.BCX,3,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F,F
2,ANA201004060,MIN,8,0,0,3,2,.CBFBBB,5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F,F
3,ANA201004060,MIN,9,1,0,0,2,.CFS,5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F,F
4,ANA201004070,MIN,9,0,0,2,2,.CBBCC,4,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F,F
5,ANA201004070,MIN,9,1,0,1,2,.BCFX,4,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,F,F
6,ANA201004080,MIN,9,0,0,0,1,.CX,6,1,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,F,F
7,ANA201004080,MIN,9,1,0,1,1,..BCX,10,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F,F
8,ANA201004090,OAK,7,0,2,1,0,.BX,10,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F,F
9,ANA201004090,OAK,9,1,1,2,2,.CB*BCFS,10,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F,F
